In [2]:
import pickle
import pandas as pd
import numpy as np


In [9]:
# Have to import the custom built class in order to load the models that use this custom class.

from sklearn import linear_model
import scipy.stats as stat

class LogisticRegression_with_p_values:
    
    def __init__(self,*args,**kwargs):#,**kwargs):
        self.model = linear_model.LogisticRegression(*args,**kwargs)#,**args)

    def fit(self,X,y):
        self.model.fit(X,y)
        
        #### Get p-values for the fitted model ####
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom,(X.shape[1],1)).T
        F_ij = np.dot((X / denom).T,X) ## Fisher Information Matrix
        Cramer_Rao = np.linalg.inv(F_ij) ## Inverse Information Matrix
        sigma_estimates = np.sqrt(np.diagonal(Cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates # z-score for eaach model coefficient
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores] ### two tailed test for p-values
        
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        #self.z_scores = z_scores
        self.p_values = p_values
        #self.sigma_estimates = sigma_estimates
        #self.F_ij = F_ij

In [10]:
# Load saved models

# Load PD model
reg_pd = pickle.load(open('pd_model.sav', 'rb'))

# Load LGD stage 1
reg_lgd_st_1 = pickle.load(open('lgd_model_stage_1.sav', 'rb'))

# Load LGD stage 2

reg_lgd_st_2 = pickle.load(open('lgd_model_stage_2.sav', 'rb'))

# Load EAD model

reg_ead = pickle.load(open('ead_model_.sav', 'rb')) 

In [14]:
# Load data to use with the PD model. Loading both train and test sets.

loan_data_inputs_train = pd.read_pickle('loan_data_inputs_train.pkl')

loan_data_inputs_test = pd.read_pickle('loan_data_inputs_test.pkl')

In [15]:
# Concatenate both dataframes into one dataframe. 
# Use axis = 0 to join by rows

loan_data_inputs_pd = pd.concat([loan_data_inputs_train, loan_data_inputs_test], axis = 0)



In [16]:
# check
loan_data_inputs_pd.shape

(466285, 322)

In [17]:
loan_data_inputs_pd.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,1,0,0,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,0,0,0,1,0,0,0,0,0,0
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,0,0,1,0,0,0,0,0,0
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,0,0,0,1,0,0,0,0,0,0
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,0,1,0,1,0,0,0,0,0,0


In [18]:
# All features used to train the PD model
features_all_pd = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>86']

In [19]:
# reference categories with the PD model
ref_categories_pd = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [20]:
# Keep only the variables used to train PD model
loan_data_inputs_pd_temp = loan_data_inputs_pd[features_all_pd]


In [21]:
# Remove the reference categories

loan_data_inputs_pd_temp = loan_data_inputs_pd_temp.drop(ref_categories_pd, axis = 1)


In [23]:
# check. Matches with the shape of dataframe used to train the PD model
loan_data_inputs_pd_temp.shape

(466285, 84)

In [25]:
# Use PD model to predict probability of default

loan_data_inputs_pd['PD'] = reg_pd.model.predict_proba(loan_data_inputs_pd_temp)[: ][: , 0]


In [26]:
#check
loan_data_inputs_pd['PD'].head()

427211    0.029585
206088    0.092148
136020    0.037359
412305    0.204330
36159     0.200844
Name: PD, dtype: float64

In [27]:
# check descriptive statistics
loan_data_inputs_pd['PD'].describe()

count    466285.000000
mean          0.109307
std           0.070917
min           0.007314
25%           0.056064
50%           0.093493
75%           0.146559
max           0.635824
Name: PD, dtype: float64

In [29]:
# Load pre-processed dataset for using with LGD and EAD models



loan_data_preprocessed = pd.read_pickle('loan_data_2007_2014_preprocessed.pkl')

In [30]:
# check

loan_data_preprocessed.head()



,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,0,0,0,0,0,0,0,1,0,1
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,0,0,0,0,0,0,1,0,0
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,0,0,0,0,0,0,1,0,1
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,0,0,0,0,0,0,1,0,1
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,0,0,0,0,0,0,1,0,1


In [28]:
#check 
loan_data_preprocessed.shape

(466285, 208)

In [31]:
# Fill missing values with 0 for mths_since_last_delinq
# Fill missing values with 0 for mths_since_last_record
# This a quick shortcut but you can do better by using a imputation technnique

loan_data_preprocessed['mths_since_last_delinq'].fillna(0, inplace = True)
loan_data_preprocessed['mths_since_last_record'].fillna(0, inplace = True)


In [33]:
#features used for LGD and EAD models

features_all = ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:MORTGAGE',
'home_ownership:NONE',
'home_ownership:OTHER',
'home_ownership:OWN',
'home_ownership:RENT',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:car',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:educational',
'purpose:home_improvement',
'purpose:house',
'purpose:major_purchase',
'purpose:medical',
'purpose:moving',
'purpose:other',
'purpose:renewable_energy',
'purpose:small_business',
'purpose:vacation',
'purpose:wedding',
'initial_list_status:f',
'initial_list_status:w',
'term_int',
'emp_length_int',
'mths_since_issue_d',
'mths_since_earliest_cr_line',
'funded_amnt',
'int_rate',
'installment',
'annual_inc',
'dti',
'delinq_2yrs',
'inq_last_6mths',
'mths_since_last_delinq',
'mths_since_last_record',
'open_acc',
'pub_rec',
'total_acc',
'acc_now_delinq',
'total_rev_hi_lim']

In [34]:
# List contianing reference variables used with LGD and EAD models
features_reference_cat = ['grade:G',
'home_ownership:RENT',
'verification_status:Verified',
'purpose:credit_card',
'initial_list_status:f']

In [35]:
# keep only variables used by EAD and LGD models

loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]


In [36]:

loan_data_preprocessed_lgd_ead = loan_data_preprocessed[features_all]# Remove reference variables

loan_data_preprocessed_lgd_ead = loan_data_preprocessed_lgd_ead.drop(features_reference_cat, axis = 1)


In [37]:
# Apply LGD stage 1 model

loan_data_preprocessed['recovery_rate_st_1'] = reg_lgd_st_1.model.predict(loan_data_preprocessed_lgd_ead)


In [38]:
# Applt LGD stage 2 model

loan_data_preprocessed['recovery_rate_st_2'] = reg_lgd_st_2.predict(loan_data_preprocessed_lgd_ead)


In [39]:
# Combine the predicted values from the stage 1 predicted model and the stage 2 predicted model
# to calculate the final estimated recovery rate.
# Same as done before: recovery_rate = recovery_rate_st_1 * recovery_rate_st_2

loan_data_preprocessed['recovery_rate'] = loan_data_preprocessed['recovery_rate_st_1'] * loan_data_preprocessed['recovery_rate_st_2']


In [40]:
loan_data_preprocessed['recovery_rate'].describe()

count    466285.000000
mean          0.079889
std           0.057382
min          -0.019612
25%           0.000000
50%           0.100884
75%           0.126446
max           0.628211
Name: recovery_rate, dtype: float64

In [41]:
# Set estimated recovery rates that are greater than 1 to 1 and  estimated recovery rates that are less than 0 to 0

loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] < 0, 0, loan_data_preprocessed['recovery_rate'])
loan_data_preprocessed['recovery_rate'] = np.where(loan_data_preprocessed['recovery_rate'] > 1, 1, loan_data_preprocessed['recovery_rate'])


In [42]:
# Compute LGD 
# LGD = 1 - recovery_rate

loan_data_preprocessed['LGD'] = 1 - loan_data_preprocessed['recovery_rate']


In [43]:
# check
loan_data_preprocessed['LGD'].describe()


count    466285.000000
mean          0.920109
std           0.057380
min           0.371789
25%           0.873554
50%           0.899116
75%           1.000000
max           1.000000
Name: LGD, dtype: float64

In [44]:
# Use EAD model to predict credit conversion factor

loan_data_preprocessed['CCF'] = reg_ead.predict(loan_data_preprocessed_lgd_ead)


In [45]:
# check
loan_data_preprocessed['CCF'].describe()

count    466285.000000
mean          0.737595
std           0.101870
min           0.292306
25%           0.671005
50%           0.730695
75%           0.803446
max           2.850761
Name: CCF, dtype: float64

In [46]:
# Compute EAD using CCF
# EAD = CCF * funded_amt

loan_data_preprocessed['EAD'] = loan_data_preprocessed['CCF'] * loan_data_preprocessed_lgd_ead['funded_amnt']


In [47]:
# check
loan_data_preprocessed['EAD'].describe()

count    466285.000000
mean      10815.272048
std        6936.974689
min         190.347372
25%        5495.101413
50%        9208.479591
75%       14692.997325
max       78059.976019
Name: EAD, dtype: float64

In [48]:
# LGD and EAD is in this dataframe
loan_data_preprocessed.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,addr_state:WY,initial_list_status:f,initial_list_status:w,good_bad,recovery_rate_st_1,recovery_rate_st_2,recovery_rate,LGD,CCF,EAD
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,0,1,0,1,1,0.086271,0.086271,0.913729,0.589922,2949.608449
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,1,0,0,1,0.084518,0.084518,0.915482,0.777773,1944.433378
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,1,0,1,1,0.080516,0.080516,0.919484,0.658306,1579.934302
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,1,0,1,1,0.095076,0.095076,0.904924,0.660656,6606.559612
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,1,0,1,1,0.088547,0.088547,0.911453,0.708211,2124.631667


In [50]:
# This data frame has the PD.
loan_data_inputs_pd.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
427211,427211,12796369,14818505,24000,24000,24000.0,36 months,8.90,762.08,A,...,0,0,1,0,0,0,0,0,0,0.029585
206088,206088,1439740,1691948,10000,10000,10000.0,36 months,14.33,343.39,C,...,0,0,1,0,0,0,0,0,0,0.092148
136020,136020,5214749,6556909,20425,20425,20425.0,36 months,8.90,648.56,A,...,0,0,1,0,0,0,0,0,0,0.037359
412305,412305,13827698,15890016,17200,17200,17200.0,36 months,16.59,609.73,D,...,0,0,1,0,0,0,0,0,0,0.204330
36159,36159,422455,496525,8400,8400,7450.0,36 months,12.84,282.40,C,...,1,0,1,0,0,0,0,0,0,0.200844


In [51]:
# check shape of both dataframes

print (loan_data_preprocessed.shape)
print (loan_data_inputs_pd.shape)

(466285, 214)
(466285, 323)


In [53]:
# Concatenate the dataframes where we calculated LGD and EAD and the dataframe where we calculated PD along the columns.
loan_data_preprocessed_new = pd.concat([loan_data_preprocessed, loan_data_inputs_pd], axis = 1)

In [54]:
# check
loan_data_preprocessed_new.shape

(466285, 537)

In [55]:
loan_data_preprocessed_new.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86,PD
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,...,1,0,1,0,0,0,0,0,0,0.164760
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,...,0,0,1,0,0,0,0,0,0,0.282340
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,...,0,0,1,0,0,0,0,0,0,0.229758
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,...,0,0,1,0,0,0,0,0,0,0.208890
4,4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,...,0,0,1,0,0,0,0,0,0,0.129554


### compute Expected loss

### Expected Loss. EL = PD * LGD * EAD.

In [56]:
loan_data_preprocessed_new['EL'] = loan_data_preprocessed_new['PD'] * loan_data_preprocessed_new['LGD'] * loan_data_preprocessed_new['EAD']


In [58]:
#check 
loan_data_preprocessed_new['EL'].describe()

count    466285.000000
mean       1075.692144
std        1091.183164
min           9.542736
25%         355.287524
50%         705.204171
75%        1395.284945
max       11909.920435
Name: EL, dtype: float64

In [59]:
# check the computed columns
loan_data_preprocessed_new[['funded_amnt', 'PD', 'LGD', 'EAD', 'EL']].head()

,funded_amnt,funded_amnt,PD,LGD,EAD,EL
0,5000,5000,0.164760,0.913729,2949.608449,444.052845
1,2500,2500,0.282340,0.915482,1944.433378,502.591459
2,2400,2400,0.229758,0.919484,1579.934302,333.775600
3,10000,10000,0.208890,0.904924,6606.559612,1248.834604
4,3000,3000,0.129554,0.911453,2124.631667,250.882460


Bank these days have to report their EL for all their loans 

They are also required by regulators to show that they have enough cash to cover their EL.

Therefore, we compute the agrregated EL for all loans


In [60]:
# Total expected loss (EL). In this case its around 501 million.

loan_data_preprocessed_new['EL'].sum()

501579111.26498187

In [61]:
# Total loan given by bank. In this case its around 6.6 billon'

loan_data_preprocessed_new['funded_amnt'].sum()

funded_amnt    6664052450
funded_amnt    6664052450
dtype: int64

### Banks are generally required to have EL to funded_amt ratio to be between 2% and 10 %.

In [67]:
el_to_amt_funded = loan_data_preprocessed_new['EL'].sum() / loan_data_preprocessed_new['funded_amnt'].sum()
print ('EL/Total funded amount', el_to_amt_funded)

EL/Total funded amount funded_amnt    0.075266
funded_amnt    0.075266
dtype: float64


### In this case the bank's EL is around 7.5% of the total loans on its book.

### Is the bank taking too much risk?

